In [45]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
movies = pd.read_csv("../input/testdataset/Content-Based-Movie-Recommendation-System-master/Content-Based-Movie-Recommendation-System-master/movie recommender dataset-20220508T165129Z-001/movie recommender dataset/movies.csv")
ratings = pd.read_csv("../input/testdataset/Content-Based-Movie-Recommendation-System-master/Content-Based-Movie-Recommendation-System-master/movie recommender dataset-20220508T165129Z-001/movie recommender dataset/ratings.csv")

In [46]:
movies.head()

In [47]:
ratings.head()
ratings = ratings.drop(columns = 'timestamp')

In [48]:
final_dataset = ratings.pivot(index='movieId',columns='userId',values='rating')
final_dataset.head()

In [49]:
final_dataset.fillna(0,inplace=True)
final_dataset.head()

In [50]:
no_user_voted = ratings.groupby('movieId')['rating'].agg('count')
no_movies_voted = ratings.groupby('userId')['rating'].agg('count')

In [51]:
f,ax = plt.subplots(1,1,figsize=(16,4))
# ratings['rating'].plot(kind='hist')
plt.scatter(no_user_voted.index,no_user_voted,color='mediumseagreen')
plt.axhline(y=10,color='r')
plt.xlabel('MovieId')
plt.ylabel('No. of users voted')
plt.show()

In [52]:
print(len(final_dataset))
final_dataset=final_dataset.loc[:,no_movies_voted[no_movies_voted > 50].index]
print(len(final_dataset))

In [53]:
csr_data = csr_matrix(final_dataset.values)
final_dataset.reset_index(inplace=True)
print(csr_data)

In [54]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(csr_data)

In [55]:
def get_movie_recommendation(movie_name):
    n_movies_to_reccomend = 10
    pred_movie_index = []
    movie_list = movies[movies['title'].str.contains(movie_name)]  
    if len(movie_list):        
        movie_idx= movie_list.iloc[0]['movieId'] # movie Id 
        movie_idx = final_dataset[final_dataset['movieId'] == movie_idx].index[0] 
        distances , indices = knn.kneighbors(csr_data[movie_idx],n_neighbors=n_movies_to_reccomend+1)    
        rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
        recommend_frame = []
        for val in rec_movie_indices:
            movie_idx = final_dataset.iloc[val[0]]['movieId']
            idx = movies[movies['movieId'] == movie_idx].index
            recommend_frame.append({'Title':movies.iloc[idx]['title'].values[0],'Distance':val[1]})
            pred_movie_index.append({'Title':movies.iloc[idx]['title'].values[0]})
            
        df = pd.DataFrame(recommend_frame,index=range(1,n_movies_to_reccomend+1))
        df2 = pd.DataFrame(pred_movie_index,index=range(1,n_movies_to_reccomend+1))
        
        print(item_evaluation(movie_idx ,df2))
        return df
    else:
        return "No movies found. Please check your input"

In [56]:
    # evaluation
def item_evaluation(movieid, predicted_movies):
        main_movie_genre = list_of_genres(movies[movies['movieId']==movieid]['genres'])
        movie_genres_list = []
        for each in predicted_movies['Title']:
            genre_list_per_movie = list_of_genres(movies[movies['title']==each]['genres'])
            movie_genres_list.append(genre_list_per_movie)
        recc_list = []
        for each in movie_genres_list:
            for all in each:
                recc_list.append(all)
        hit = 0
        fault = 0
        total = 0
        for i in recc_list:
            if i in main_movie_genre:
                hit += 1
            else:
                fault += 1
            total += 1
        return hit/total

In [57]:
# Collect the unique set of all the genres
def list_of_genres(df):
    genre_list = []
    for i in range(len(df)):
        each_genre = df.iloc[i]
        string_split = each_genre.split("|")
        #     print(string_split)
        for each in string_split:
            if each not in genre_list:
                genre_list.append(each)
    return genre_list

In [60]:
get_movie_recommendation('Lion King')